In [5]:
import face_recognition
import cv2
from datetime import datetime
import json
import os
import numpy as np

# Path to the face database
database_path = "face_data.json"

# 1. Encode initial images and store them with additional data
def encode_images(image_folder):
    face_data = {}
    
    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.jpg', '.png')):
            image_path = os.path.join(image_folder, image_name)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            
            if encoding:
                name = os.path.splitext(image_name)[0]
                face_data[name] = {
                    "encoding": encoding[0].tolist(),
                    "phone": "123456789",
                    "email": f"{name}@example.com"
                }
    
    with open(database_path, 'w') as f:
        json.dump(face_data, f)
    
    print("All images have been encoded and saved.")

# 2. Add a new user to the database
def add_new_user(image_path, user_name, phone, email):
    image = face_recognition.load_image_file(image_path)
    encoding = face_recognition.face_encodings(image)
    
    if encoding:
        try:
            with open(database_path, 'r') as f:
                face_data = json.load(f)
        except FileNotFoundError:
            face_data = {}

        face_data[user_name] = {
            "encoding": encoding[0].tolist(),
            "phone": phone,
            "email": email
        }
        
        with open(database_path, 'w') as f:
            json.dump(face_data, f)
        
        print(f"User {user_name} added successfully.")
    else:
        print("No face found in the image.")

# 3. Run real-time face recognition and record attendance
attendance_report = {}

def load_database():
    with open(database_path, 'r') as f:
        face_data = json.load(f)
    
    known_names = list(face_data.keys())
    known_encodings = [np.array(face_data[name]["encoding"]) for name in known_names]
    user_info = {name: {"phone": face_data[name]["phone"], "email": face_data[name]["email"]} for name in known_names}
    
    return known_names, known_encodings, user_info

def recognize_faces_in_real_time():
    known_names, known_encodings, user_info = load_database()
    video_capture = cv2.VideoCapture(0)
    
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
        
        for face_encoding, face_location in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_encodings, face_encoding)
            name = "Unknown"
            
            if True in matches:
                first_match_index = matches.index(True)
                name = known_names[first_match_index]
                
                if name not in attendance_report:
                    attendance_report[name] = {
                        "time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        "phone": user_info[name]["phone"],
                        "email": user_info[name]["email"]
                    }
                    print(f"{name} attended. Details: Phone - {user_info[name]['phone']}, Email - {user_info[name]['email']}")

            # Draw a rectangle around the face and display the name
            top, right, bottom, left = face_location
            top, right, bottom, left = top * 2, right * 2, bottom * 2, left * 2
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
        
        cv2.imshow('Face Recognition', frame)
        
        # Press 'r' to reload the database, or 'q' to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord('r'):
            known_names, known_encodings, user_info = load_database()
            print("Database reloaded.")
        elif key == ord('q'):
            break
    
    video_capture.release()
    cv2.destroyAllWindows()
    save_attendance_report()

def save_attendance_report(report_path="attendance_report.json"):
    with open(report_path, 'w') as f:
        json.dump(attendance_report, f)
    
    print("Attendance report saved.")

# Usage steps:
# 1. Encode initial images (run once)
# encode_images("path/to/image_folder")

# 2. Add a new user
# add_new_user("path/to/new_image.jpg", "User Name", "123456789", "user@example.com")

# 3. Run face recognition and record attendance
# recognize_faces_in_real_time()


In [6]:
encode_images("Faces/")

All images have been encoded and saved.


In [7]:
add_new_user("new/Eng Ahmed Hany.jpg", "ahmed hany", "123456789", "ahmed@example.com")

User ahmed hany added successfully.


In [8]:
recognize_faces_in_real_time()

Bola Emad attended. Details: Phone - 123456789, Email - Bola Emad@example.com
Database reloaded.
Attendance report saved.
